In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from weight_discovery import *

In [ ]:
from kyle_tools import rc_dict
rc_dict['legend.loc'] = 'upper right'
rc_dict['legend.fontsize'] = 12

for k,v in rc_dict.items():
    plt.rcParams[k] = v

In [ ]:
files = [f'./data/jinghao_mystery_traj_dt_p001_ep_p25_{n}.npy' for n in [f'{N:02}' for N in range(11,14)]]

process = np.array([np.load(item) for item in files]).squeeze()

d=3

s =  process.shape

new_len = int(s[2]/d)
new_process = np.empty( (s[0],d*s[1],new_len,s[-1]) )
for i in range(d):
    new_process[:,i*s[1]:(i+1)*s[1],:,:] = process[:,:,i*new_len:(i+1)*new_len ,:]
    
new_process = new_process.reshape(d*s[0]*s[1], new_len,s[-1])

print(process.shape, new_process.shape)
process = new_process


In [ ]:
'''
process = np.array(np.load('./PWL_UD_NESS_dt_p005_ep_p55_01.npy')).squeeze()

d=25
s =  process.shape

new_len = int(s[1]/d)

new_process = np.empty( (d*s[0],new_len,s[-1]) )
for i in range(d):
    new_process[i*s[0]:(i+1)*s[0],:,:] = process[:,i*new_len:(i+1)*new_len ,:]


print(process.shape, new_process.shape)
process = new_process


In [ ]:
files = [f'./data/jinghao_mystery_traj_dt_p001_ep_p25_{n}.npy' for n in [f'{N:02}' for N in range(5,15)]]

fles = [ [ files[i], files[i+1] ] for i in range(0,len(files)-1,2) ]

In [ ]:
fig, ax = plt.subplots(len(files),2 , figsize=(10,20), sharex='col', sharey='col')

for i,f in enumerate(files):
    td = np.load(f)[...,1].ravel()
    td1 = td[td>4]
    td2 = td[td<-4]
    ax[i,0].set_title(f'{len(td2)} {len(td1)}')
    ax[i,0].hist(td2)
    ax[i,1].hist(td1)
    #ax[i].axvline(-4.9)
    #ax[i].set_title(f'max:{np.max(td1)} min:{np.min(td)}')

In [ ]:
fles

In [ ]:
ep = []
epmin = []
epmax = []

sample_list = []

#for fls in fles:
#        process = np.vstack(np.array([np.load(item) for item in fls]).squeeze())
#        sp = process.shape
#        d=3
#        nl, ns = sp[0]*d, int(sp[1]/d)
#        new_proc = np.empty( (nl, ns, sp[-1]) )
#        for i in range(d):
#                new_proc[i*sp[0]:(i+1)*sp[0],...] = process[:,i*ns:(i+1)*ns,:]
#        process = new_proc
print(process.shape)
# for calcualting EPR
dt=1/5000
step_skip=5
cg = 1
K = 1
samples = 20_000_000
dx_state = process[:samples,::cg,0]
cyclic_lims = [-1.5,1.5]
dx = np.diff(dx_state, axis=1)
if cyclic_lims is not None:
        domain = np.diff(np.asarray(cyclic_lims))
        jump_back = dx < -.8*domain
        jump_forw = dx > .8*domain
        dx[jump_back] = dx[jump_back] + domain
        dx[jump_forw] = dx[jump_forw] - domain
        ast = K*dx
ast_d = ast/(cg*dt*step_skip)
ent = ast_d.mean(axis=0)
err = 3*ast_d.std(axis=0)/np.sqrt(len(process))
ep.append(ent.mean())
epmin.append(np.min(ent-err))
epmax.append(np.max(ent+err))



ax.errorbar( range(len(ast_d[0,:])), ent, yerr=err, linestyle='none', marker='D' );
ax.axhline(ent.mean())
print(ent.mean())

In [ ]:
plt.plot(ep, marker='o')
plt.plot(epmax)
plt.plot(epmin)

In [ ]:
plt.plot(ep, marker='o')
plt.plot(epmax)
plt.plot(epmin)

In [ ]:
plt.plot(ep, marker='o')
plt.plot(epmax)
plt.plot(epmin)

* 10M : 0.2542; .253, .256
* 8M : .2542; .253, .256
* 5 M: .2535; .252, .255

In [ ]:
fig, ax  = plt.subplots()

pcg = 1

ax.errorbar( range(len(ast_d[0,::pcg])), ent[::pcg], yerr=3*err[::pcg]/np.sqrt(len(process[:samples])), linestyle='none', marker='D' );
ax.axhline(ent.mean())

In [ ]:
process.shape

In [ ]:
num_est = 15
time_step = 100
gamma = 1
proc_dt = .001
kwargs = {'cyclic_lims':[-1.5,1.5]}



In [ ]:
ep = m@np.linalg.inv(X)@m

In [ ]:
ep/proc_dt

In [ ]:
ep/proc_dt

In [ ]:
xr = np.sort(X.ravel())
X[X<xr[int(len(xr)*.15)]] = 0

In [ ]:
fig, ax = plt.subplots()
ax.plot(xr[:40_000])
ax.set_yscale('log')
ax.set_ylim(1e-30,np.max(xr))

In [ ]:
fig, ax = plt.subplots()
ax.set_xscale('log')

ax.hist(X[X<1e-16].ravel())

In [ ]:

num_est = 15

#steps = [1, 3, 5, 15, 30, 100]
steps = np.arange(0,50,1)

mus = []
Xis = []
for time_step in steps:
    mu,Xi = get_step_epr(process[:1_200_000], time_step, proc_dt, gamma, num_est, return_mats=True, **kwargs)
    mus.append(mu)
    Xis.append(Xi)




In [ ]:
mu_avg = np.mean( np.array(mus), axis=0)
Xi_avg = np.mean( np.array(Xis), axis=0)


In [ ]:
tep = []
for mu in mus:
    tep.append((mu@np.linalg.inv(Xi_avg)@mu)/proc_dt)

In [ ]:
(mu_avg@np.linalg.inv(Xi_avg)@mu_avg)/proc_dt

In [ ]:
(mu_avg@np.linalg.inv(Xi_avg)@mu_avg)/proc_dt

In [ ]:
np.max(epsl_drop)

In [ ]:
fig, ax = plt.subplots()

ax.scatter(range(len(epsl)), epsl)
ax.scatter(range(len(epsl_drop)), epsl_drop)

colors = ['g','y' 'b']
for i,eps in enumerate([tep]):
    m, s  = np.mean(eps), 3*np.std(eps)/np.sqrt(len(eps))
    
    ax.scatter(range(len(eps)), eps, label='trial estimates', c=colors[i], alpha=.2)
    ax.axhline(m, c=colors[i], label='estimated average EPR')
    ax.axhline(m+s, c=colors[i], linestyle='--', label='99% Estimation Confidence Interval')
    ax.axhline(m-s, c=colors[i], linestyle='--')
ax.axhline(.2542, c='k', label='calculated average EPR')

fig.legend()


In [ ]:
epsr2 = []  
num_est = 13
n_samples = 2_000_000
n_iter = 50

  
steps = range(process.shape[1]-2)

steps_idx = np.meshgrid(steps, np.empty(n_samples))[0]
steps_idx = np.random.default_rng().permuted(steps_idx,axis=1)[:,:50]

for time_step in steps:

        print(f'{time_step}', end='\r')
        proc = np.empty( (n_samples,3,2) )
        for j in range(3):
            proc[:,j] = process[range(n_samples),steps_idx[:,0]+j]

        epsr2.append(get_step_epr(proc, 0, proc_dt, gamma, num_est, **kwargs ))
        steps_idx = steps_idx[:,1:]

In [ ]:
epsr=[]
n_samples = 2_000_000
n_iter = 50

num_est = 13
n_steps = process.shape[1]

for i in range(n_iter):
    print(f'{i}', end='\r')
    temp_proc = np.empty( (n_samples,4,2) )
    steps = np.random.randint(0, n_steps-3, n_samples)

    for j in range(4):
        temp_proc[:,j] = process[range(n_samples),steps+j]
    tep = get_step_epr(temp_proc, 0, proc_dt, gamma, num_est, **kwargs )
    epsr.append(tep)
    if tep > 1:
        print(np.max(steps), np.min(steps))





In [ ]:
fl

In [ ]:
'Xis' in data[24].keys()

In [ ]:
x=np.array([3,3,4])
len(np.where(x>4)[0])

In [ ]:
#fl = kt.file_list('data/cyclic_ness_estimates/shuffle_pad/holding cell/')
fl = kt.file_list('data/cyclic_ness_estimates/regularizer/')

#fl.extend(kt.file_list('data/cyclic_ness_estimates/shuffle_pad/'))
fl.sort()

data = [kt.open_json(i) for i in fl]

data = [D for D in data if 'Xis' in D.keys()]
#del(data[-2])

In [ ]:
outliers = False


idxs = range(len(data))
for i in idxs:

    #weird data mystery
    #if len(data[i]['data']) != len(data[i]['Xis']):
    if True:
        EPL = []
        for x,m in zip(data[i]['Xis'], data[i]['mus']):
            EPL.append( (m @ np.linalg.inv(x) @ m)/data[i]['dt'] )
        data[i]['data'] = EPL

    #weird outliers mystery
    if outliers:
        eps = np.array(data[i]['data'])
        mean, std_dev, median = np.mean(eps), np.std(eps), np.median(eps)

        objective = np.where( np.abs(eps-mean)>.6*std_dev)
        objective = np.where( eps>5)
        data[i]['data'] = np.delete(eps, objective)
        if len(objective[0])>0:
            print(len(objective[0]))



In [ ]:
fl

In [ ]:
data[24]['num_path']

In [ ]:
np.where(E==10)

In [ ]:
plt.plot(data[24]['data'])
plt.plot(data[24]['reg_data'])

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
for X, RX in zip(data[24]['Xis'][4:5],data[24]['reg_Xis'][4:5]):
    item = (RX.ravel()- X.ravel())/X.ravel()
    plt.plot( item[::1] )
    print(np.min(item), np.max(item), np.mean(item), np.sum(item))


ax.set_yscale('log')

In [ ]:
(1E-7)/(5.4E6)

In [ ]:
data[5]['Xis'][0].shape

In [ ]:
EPL = []



In [ ]:
np.mean(EPL)

In [ ]:
for i in range(len(data)):
    print(i)
    print(np.mean(data[i]['data']))

In [ ]:
for i in range(len(data)):
    print(i)
    print(np.mean(data[i]['data']))

In [ ]:
S = data[24]['Xis'][0].size

sorted(data[24]['Xis'][0].ravel())[ -int(S/4)] /(5.4E6)




In [ ]:
x = np.linspace(1, 1_000, 10_000_000)
plt.semilogy(1/x)
plt.semilogy(x**-.5)
plt.semilogy(x**-.75)
#plt.semilogy(x**2)


In [ ]:


for i in range(len(data)):

    def alpha(data, N = data[i]['num_path']):
        S = data.size
        return 1/N**(2)

    add_regularizer(data[i], .001, alpha_gen = alpha, verbose=True )
    

In [ ]:
key = 'reg_data'
m  = np.array([np.mean(i[key]) for i in data])
max = np.array([np.max(i[key]) for i in data])
min = np.array([np.min(i[key]) for i in data])
s = np.array([ 3*np.std(i[key])/np.sqrt(len(i['data'])) for i in data])

N = np.array([i['num_path'] for i in data])
E = np.array([i['num_est'] for i in data])

Nvals = list(set(N))
Nvals.sort()

Evals = list(set(E))
Evals.sort()

In [ ]:
print( np.mean(data[24]['reg_data']), np.std(data[24]['reg_data']))

In [ ]:
print( np.mean(data[24]['reg_data']), np.std(data[24]['reg_data']))

In [ ]:
%debug

In [ ]:
    fig, ax = plt.subplots(1,2, sharex=True, sharey=True)
    for d in data:
        eps = np.array(d['data'])
        ax[0].plot(d['data'], linestyle='none', marker='o')
        ax[1].plot(d['reg_data'], linestyle='none', marker='o')

In [ ]:
np.array([i['num_est'] for i in data])

In [ ]:
fig, ax = plt.subplots()
ax.plot(data[-1]['data'])
ax.plot(data[-2]['data'])
ax.set_ylim(0,3)

In [ ]:
plt.plot(data[-1]['data'])

In [ ]:
pd = data[0]
max_idx = np.argmax(pd['data'])
fig, ax = plt.subplots(2)

Xi_avg = np.linalg.inv((np.array(pd['Xis'])[np.where(np.array(pd['data']) < .7)].mean(axis=0)))
Xi_odd = np.linalg.inv((pd['Xis'][max_idx]))
Xi_norm = np.linalg.inv((pd['Xis'][5]))

Xi_i = []
for i in np.where(np.array(pd['data']) < 100)[0]:
    Xi_i.append(np.linalg.inv(pd['Xis'][i]))

Xi_i_avg = np.array(Xi_i).mean(axis=0)

mu_avg = np.array(pd['mus'])[np.where(np.array(pd['data']) < .7)].mean(axis=0)
mu_odd = (pd['mus'][max_idx])
mu_norm = (pd['mus'][5])

print('Xi')
print(np.min(Xi_avg), np.max(Xi_avg), np.mean(Xi_avg))
print(np.min(Xi_odd), np.max(Xi_odd), np.mean(Xi_odd))
print(np.min(Xi_norm), np.max(Xi_norm), np.mean(Xi_norm))
print('mu')
print(np.min(mu_avg), np.max(mu_avg), np.mean(mu_avg))
print(np.min(mu_odd), np.max(mu_odd), np.mean(mu_odd))


for i in range(30):
    mu_norm = (pd['mus'][i])
    Xi_norm = np.linalg.inv((pd['Xis'][i]))

    ax[0].plot(  np.sort((Xi_norm @ mu_norm)), c='b', alpha=.1)
    ax[1].plot(  np.sort(Xi_norm.ravel()), c='b', alpha=.1)

c1 = ax[0].plot( np.sort((Xi_odd @ mu_odd)), c='r' ) 
c1 = ax[0].plot( np.sort((Xi_avg @ mu_avg)), c='k' ) 
#c1 = ax[0].plot( np.sort((Xi_i_avg @ mu_avg)), c='y' ) 

c1 = ax[1].plot( np.sort(Xi_odd.ravel()), c='r' ) 
c1 = ax[1].plot( np.sort(Xi_avg.ravel()), c='k' ) 
#c1 = ax[1].plot( np.sort(Xi_i_avg.ravel()), c='y' ) 

#fig.colorbar(c0, ax=ax[0])
#fig.colorbar(c1, ax=ax[1])

#ax[0].set_ylim(50,200)
#ax[0].set_xlim(50,200)

ax[0].set_yscale('symlog')
ax[1].set_yscale('symlog')




In [ ]:
data[0]['Xis']

In [ ]:
np.mean(data[0]['Xis'], axis=0).shape

In [ ]:
for j,d in enumerate(data):
    
    try: X = np.mean(d['Xis'], axis=0)
    except: continue
    print(f'{j}')
    #xr = np.sort(X.ravel())
    #X[X<xr[int(len(xr)*.20)]] = 0
    for i in range(len(d['data'])):
        mu = np.array(d['mus'][i])
        data[j]['Xis'][i] = X
        data[j]['data'][i] = (mu @ np.linalg.inv(X) @ mu)/d['dt']

m  = np.array([np.mean(i['data']) for i in data])
max = np.array([np.max(i['data']) for i in data])
min = np.array([np.min(i['data']) for i in data])
s = np.array([ 3*np.std(i['data'])/np.sqrt(len(i['data'])) for i in data])

N = np.array([i['num_path'] for i in data])
E = np.array([i['num_est'] for i in data])

Nvals = list(set(N))
Nvals.sort()

Evals = list(set(E))
Evals.sort()


In [ ]:
fig, ax = plt.subplots()
for ev in Evals:
    ax.plot(N[E==ev],max[E==ev], marker='D', label=f'{ev}')
    fig.legend()
    #ax.set_yscale('log')
    #ax.set_ylim(.2,1)


In [ ]:
rc_dict['legend.loc'] = 'upper right'
for k,v in rc_dict.items():
    plt.rcParams[k] = v

idx=39

eps = np.array(data[idx]['data'])
d = data[idx]

fig, ax = plt.subplots(figsize=(5*1.2,3*1.2))



m, s  = np.mean(eps), 3*np.std(eps)/np.sqrt(len(eps))

ax.scatter(range(len(eps)), eps, marker='o', s=15, label='trial estimates')
ax.axhline(m, c='r', label='estimated average EPR')

#ax.axhline(.5506, c='k', label='average EPR')
#ax.axhline(.2535, c='k', label='average EPR')
ax.axhline(.2542, c='k', label='calculated average EPR')
#ax.axhline(.2546, c='k', label='calculated average EPR')


ax.axhspan(.252, .256, facecolor='k', alpha=0.3, label='finite size EPR variance')
#ax.axhspan(.248, .256, facecolor='k', alpha=0.3, label='finite size EPR variance')

#ax.axhspan(.252, .255, facecolor='k', alpha=0.3, label='finite size EPR range')
#ax.axhspan(.253, .256, facecolor='k', alpha=0.3, label='finite size EPR range')

print(m)
ax.axhline(m+s, c='r', linestyle='--', label='99.7% Estimation Confidence Interval')
ax.axhline(m-s, c='r', linestyle='--')
fig.legend()

ax.set_xlabel('trial')
ax.set_ylabel('EPR')


#ax.set_ylim(.54,.56)

#fig.suptitle(f'{d["num_est"]**2} estimators, {d["num_path"]/1e6:.3} Million Trajectories ')

ax.set_ylim(.225,.29)
#ax.set_yscale('log')



In [ ]:
fig.savefig('NESS_E18N108.pdf')

In [ ]:
E==15

In [ ]:
Nvals = list(set(N))
Nvals.sort()

N_axes = {N:i for N,i in zip(Nvals, range(len(Nvals)))}

Evals = list(set(E))
Evals.sort()



fig, ax = plt.subplots(1, figsize=(5*1,4*1), sharex=True, sharey=True )

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plot_kwargs = {'linewidth':1, 'linestyle':'--','marker':'D','markersize':3, 'elinewidth':4, 'alpha':.7 }

estimated_epr = .2535

for i,n in enumerate(Evals[:]):
    if n in [8, 12, 14, 16, 18, 21]:
        msk = np.where(E==n)

        trim = 0
        mtemp = m[msk][trim:]
        stemp = s[msk][trim:]
        Ntemp = N[msk][trim:]
        #nlst = [n+(.8)*(n**.7)*(i-((sum(N==n)-1)/2)) for n in Ntemp]
        nlst = [n+n*(.01)*(i-((sum(N==n)-1)/2)) for n in Ntemp]

        #mtemp = (mtemp-estimated_epr)/estimated_epr
        #stemp = stemp/estimated_epr
        ax.errorbar(nlst, mtemp, yerr = stemp, **plot_kwargs , label=f'{n**2} estimators')

        mv_prev= None
        for mv, sv, nv in zip(mtemp,stemp, Ntemp):
            ax_i = N_axes[nv]
        
ax.axhline(estimated_epr, label='$\Sigma/dt$', c='k', linewidth=1, alpha=1)
#ax.axhline(.2546, c='k', label='calculated average EPR')

#ax.axhline(.2388, label='average EPR', c='r', linewidth=1, alpha=1)

#ax.axhspan(.252, .255, facecolor='k', alpha=0.3, label='EPR finite size variance')
#ax.axhspan(.252-estimated_epr, .255-estimated_epr, facecolor='k', alpha=0.3, label='EPR finite size variance')

#ax.axhspan(.248, .256, facecolor='k', alpha=0.3, label='finite size EPR variance')


            
fig.legend()

ax.set_xlabel('N')
ax.set_ylabel('$\hat{\Sigma}/dt$')
ax.set_xscale('log')
#ax.set_xticks([500_000, 1_000_000, 5_000_000])

#ax.set_ylim(-100, 250)

ax.set_ylim(.20,.75)

#ax.set_ylim(.23,.27)

#ax.set_xlim(10_000, 12_000_000)
#ax.axvline(2_000_000)


#ax.set_xlim(400_000, 600_000)
#ax.axvline(500_000)

#ax.axhline(.254)
#ax.axhline(.251)
#ax.axhline(.246)

#ax.axvline(8_000_000)
#ax.axvline(5_000_000)

In [ ]:
fig.savefig('cyclic_ness_reg.pdf')

In [ ]:
from matplotlib.patches import ConnectionPatch as connPatch
Nvals = list(set(N))
Nvals.sort()

N_axes = {N:i for N,i in zip(Nvals, range(len(Nvals)))}

Evals = list(set(E))
Evals.sort()

fig, ax = plt.subplots(1,len(Nvals), figsize=(10,6), sharex=True, sharey=True )

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

for i,n in enumerate(Evals):
    msk = np.where(E==n)
    mtemp = m[msk]
    stemp = s[msk]
    Ntemp = N[msk]

    cv = colors[i]
    mv_prev= None
    for mv, sv, nv in zip(mtemp,stemp, Ntemp):
        ax_i = N_axes[nv]
        ax[ax_i].errorbar(n**2, mv, yerr = sv, linestyle='none', marker='D', c=cv)
        if mv_prev != None:
            connect = connPatch(xyA=(n**2,mv_prev) ,xyB=(n**2,mv) ,coordsA="data", coordsB="data", axesA=ax[ax_i-1], axesB=ax[ax_i], color=cv )
            #ax[ax_i].add_artist(connect)
        mv_prev = mv
    
for i,n in enumerate(Nvals):
    ax[i].set_title(f'N= {n:.1e}')
    ax[i].axhline(.2535, label='exact EPR', c='k', linewidth=1, alpha=.4)
    ax[i].axhspan(.252, .255, facecolor='k', alpha=0.3)


            


ax[0].set_xlabel('num est')
ax[0].set_ylabel('EPR')
ax[0].set_xscale('log')
#ax[0].set_ylim(.15,.35)

fig.tight_layout()
#ax.scatter([s*proc_dt for s in steps], eps, marker='D')

In [ ]:
Nvals = list(set(N))
Nvals.sort()

Evals = list(E)
Evals.sort()

fig, ax = plt.subplots(1,len(Nvals), figsize=(9,6), sharex=True, sharey=True )

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

for i,n in enumerate(Nvals):
    msk = np.where(N==n)
    mtemp = m[msk]
    stemp = s[msk]
    etemp = E[msk]
    c = colors[:len(m[msk])]
    j=0
    for mv, sv, ev, cv in zip(mtemp,stemp, etemp, c):
        ax[i].errorbar(ev**2, mv, yerr = sv, linestyle='none', marker='D')
        ax[i].axhline(m[N==Nvals[-1]][j], linestyle='--', c=cv, alpha=.5)
        j+=1
    ax[i].set_title(f'N= {n:.1e}')
    ax[i].axhline(.2535, label='exact EPR', c='k', linewidth=1, alpha=.4)
    ax[i].axhspan(.252, .255, facecolor='k', alpha=0.3)


            


ax[0].set_xlabel('num est')
ax[0].set_ylabel('EPR')
ax[0].set_xscale('log')
ax[0].set_ylim(.15,.35)


#ax.scatter([s*proc_dt for s in steps], eps, marker='D')

In [ ]:
fig.savefig('cyclic_ness_scaling_2.pdf')